In [2]:
import pandas as pd
import plotly.express as px
import chart_studio.plotly as py
import chart_studio.tools as tls
import ipywidgets as widgets
from IPython.display import display, clear_output

In [3]:
data = pd.read_excel(r"\\corp.hertshtengroup.com\Users\India\Data\tirtha.nandy\Documents\#Data\FCPO\Cash Market\FCPO Cash Market.xlsx")
data_clean = data[['FCPO Cash', 'Last', 'FCPOc1', 'Last.1', 'FCPOc2', 'Last.2', 'FCPOc3', 'Last.3']]
data_clean.columns = ['Date_Cash', 'Cash', 'Date_C1', 'C1', 'Date_C2', 'C2', 'Date_C3', 'C3']

merged_data = data_clean.merge(data_clean[['Date_C1', 'C1', 'C2', 'C3']], 
                               left_on='Date_Cash', right_on='Date_C1', 
                               how='right')

merged_data = merged_data[['Date_C1_y', 'Cash', 'C1_y', 'C2_y', 'C3_y']]
merged_data.columns = ['Date', 'Cash', 'C1', 'C2', 'C3']

tls.set_credentials_file(username='Nyctoris', api_key='VZipMyuR3UMvXmRjyTbm')


In [4]:
def create_initial_plot():
    correlation_matrix = merged_data[['Cash', 'C1', 'C2', 'C3']].corr()

    fig = px.imshow(correlation_matrix, 
                    text_auto=True, 
                    aspect="auto", 
                    title="Correlation Matrix: Cash vs C1 vs C2 vs C3",
                    color_continuous_scale='Viridis')

    fig.update_layout(height=600, width=800, template='plotly_white')
    return fig

def update_plot(fig, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    filtered_data = merged_data[(merged_data['Date'] >= start_date) & (merged_data['Date'] <= end_date)]

    correlation_matrix = filtered_data[['Cash', 'C1', 'C2', 'C3']].corr()

    fig.data[0].z = correlation_matrix.values
    fig.data[0].zmin = correlation_matrix.values.min()
    fig.data[0].zmax = correlation_matrix.values.max()
    fig.update_layout(title=f"Correlation Matrix: Cash vs C1 vs C2 vs C3 ({start_date.date()} to {end_date.date()})")
    
    clear_output(wait=True)
    display(start_date_picker, end_date_picker)
    fig.show()


fig = create_initial_plot()
plot_url = py.plot(fig, filename='interactive_correlation_matrix', auto_open=True)

start_date_picker = widgets.DatePicker(
    description='Start Date',
    value=pd.to_datetime('2019-08-26').date(),
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    value=pd.to_datetime('2024-08-23').date(),
    disabled=False
)

def on_date_change(change):
    update_plot(fig, start_date_picker.value, end_date_picker.value)

start_date_picker.observe(on_date_change, names='value')
end_date_picker.observe(on_date_change, names='value')

display(start_date_picker, end_date_picker, fig)

plot_url_updated = py.plot(fig, filename='interactive_correlation_matrix_updated', auto_open=False)
print(f"Interactive plot available at: {plot_url}")

DatePicker(value=datetime.date(2019, 8, 26), description='Start Date')

DatePicker(value=datetime.date(2024, 8, 23), description='End Date')

Interactive plot available at: https://plotly.com/~Nyctoris/1/
